In [ ]:
from bokeh.core.json_encoder import serialize_json
from bokeh.embed.elements import div_for_render_item
from bokeh.embed.util import RenderItem
from bokeh.io import curdoc, output_notebook
from bokeh.io.notebook import get_comms, publish_display_data
from bokeh.protocol import Protocol
from bokeh.util.serialization import make_id

template = '''
{
    const { Document } = Bokeh.require('document');
    const { notebook: { kernel: { comm_manager } } } = Jupyter;
    const doc = Document.from_json(%s);
    const { elementid, roots } = %s;
    const target = '%s';

    comm_manager.register_target(target, (comm) => {
        comm.on_msg(({ content: { data } }) => {
            doc.apply_json_patch(data);
        });
        doc.on_change((event) => {
            if (event.setter_id === target) {
                return;
            }
            comm.send(doc.create_json_patch([event]));
        });
    });
    
    const element = document.getElementById(elementid);
    const root_ids = Object.entries(roots)
        .reduce((obj, [key, id]) => ({ ...obj, [key]: document.getElementById(id) }), {});
    Bokeh.embed.add_document_standalone(doc, element, root_ids);
}'''

def show(model):
    doc = curdoc()
    doc.clear()
    doc.add_root(model)
    
    comms_target = make_id()
    
    render_item = RenderItem(make_id(), elementid=make_id(), roots=[model])
    div = div_for_render_item(render_item)
    script = template % (serialize_json(doc.to_json()),
                         serialize_json(render_item.to_json()),
                         comms_target)
    
    HTML_MIME_TYPE = 'text/html'
    JS_MIME_TYPE   = 'application/javascript'
    EXEC_MIME_TYPE = 'application/vnd.bokehjs_exec.v0+json'

    publish_display_data({HTML_MIME_TYPE: div})
    publish_display_data({JS_MIME_TYPE: script, EXEC_MIME_TYPE: ""},
                         metadata={EXEC_MIME_TYPE: {"id": model._id}})
    
    comms = get_comms(comms_target)
    protocol = Protocol('1.0')
    
    @doc.on_change
    def send_patch_doc(event):
        message = protocol.create('PATCH-DOC', [event])
        comms.send(message.content)

    @comms.on_msg
    def handle_msg(msg):
        doc.apply_json_patch(msg['content']['data'])

output_notebook()

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure

x = [t * .005 for t in range(200)]
source = ColumnDataSource(data=dict(x=x, y=x))

def callback(attr, old, new):
    source.data = dict(x=x, y=[t**new for t in x])

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.on_change('value', callback)

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

model = column(slider, plot)
show(model)